# Install and import dependencies

In [1]:
!pip install mediapipe opencv-python

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/ed/b1/978712efb3c370eca2d4224d1cac4582561f594eb759ba1ed4a082e9a704/mediapipe-0.10.8-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=2.0 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/81/3c/bbb3ceee9fbefc505f98c24dafda68c7b3c4f

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Set up media pipe
mp_drawing = mp.solutions.drawing_utils
# Import pose estimation model
mp_pose = mp.solutions.pose

In [2]:
# VIDEO FEED

# Open the video file
video_path = r'videos\1_backhand.mp4'
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    # Read a frame from the video
    # ret -> return variable
    # frame -> image from cap
    ret, frame = cap.read()
    
    # Display the frame with results
    cv2.imshow('UltiThrow', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

# Make detections

mp_pose.Pose()
min_detection_confidence
min_tracking_confidence

pose

In [2]:
# Open the video file
video_path = r'videos\1_backhand.mp4'
cap = cv2.VideoCapture(0)

# Get the original width and height of the video
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("End of video.")
            break  # Exit the loop when the video is finished.
        

        # Detect stuff and render
        
        # Recolor image
        # default of opencv is rgb, that's why we recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # set the image to not be modified and save memory
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image) # get detection of pose
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        # draw_landamarks -> utils to draw in image
        # results.pose_landmarks -> coordenates
        # mp_pose.POSE_CONNECTIONS -> how body points are conencted inside model
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Update desired height and width before resizing
        desired_height, desired_width = original_height // 2, original_width // 2

        # Resize the image
        image = cv2.resize(image, (desired_width, desired_height))

        # Display the resized image
        cv2.imshow('UltiThrow', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()

# Extract landmarks

In [25]:
# Open the video file
video_path = r'videos\1_backhand_front.mp4'
cap = cv2.VideoCapture(video_path)

# Get the original width and height of the video
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("End of video.")
            break  # Exit the loop when the video is finished.
        

        # Detect stuff and render
        
        # Recolor image
        # default of opencv is rgb, that's why we recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # set the image to not be modified and save memory
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image) # get detection of pose
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Render detections
        # draw_landamarks -> utils to draw in image
        # results.pose_landmarks -> coordenates
        # mp_pose.POSE_CONNECTIONS -> how body points are conencted inside model
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Update desired height and width before resizing
        desired_height, desired_width = original_height // 2, original_width // 2

        # Resize the image
        image = cv2.resize(image, (desired_width, desired_height))

        # Display the resized image
        cv2.imshow('UltiThrow', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()

In [14]:
len(landmarks)

33

In [21]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [26]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value] # how to access the values

x: 0.50659496
y: 0.36798444
z: -0.059700783
visibility: 0.96365255

Extracting information about backhand from a rigth-hand person video taken from front perspective.

12 - right shoulder
14 - right elbow
16 - right wrist

In [27]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

x: 0.4252391
y: 0.26356584
z: -0.052073795
visibility: 0.99997765

In [28]:
landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

x: 0.40820667
y: 0.3729021
z: -0.01889005
visibility: 0.98281825

In [29]:
landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

x: 0.38858938
y: 0.4651661
z: -0.08842416
visibility: 0.9647229

# Calculate angles

In [25]:
def calculate_angle(a, b, c): #first, mid and end points
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0] - b[0]) - np.arctan2(a[1]-b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [8]:
shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

In [37]:
shoulder, elbow, wrist

([0.42523908615112305, 0.2635658383369446],
 [0.42523908615112305, 0.2635658383369446],
 [0.42523908615112305, 0.2635658383369446])

In [39]:
calculate_angle(shoulder, elbow, wrist)

0.0

# Working angle for 2D

In [48]:
# Open the video file
video_path = r'videos\0_backhand_front.mp4'
cap = cv2.VideoCapture(video_path)

# Get the original width and height of the video
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Update desired height and width before resizing
desired_height, desired_width = original_height // 2, original_width // 2
#desired_height, desired_width = original_height, original_width
# Set up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("End of video.")
            break  # Exit the loop when the video is finished.
        

        # Detect stuff and render

        # Resize the image
        frame = cv2.resize(frame, (desired_width, desired_height))
        
        # Recolor image
        # default of opencv is rgb, that's why we recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # set the image to not be modified and save memory
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image) # get detection of pose
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # calculate angle
            angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)
            angle_left = calculate_angle(shoulder_left, elbow_left, wrist_left)
            
            # display value
            cv2.putText(image, str(round(angle_right, 2)), 
                        tuple(np.multiply(elbow_right, [desired_width, desired_height]).astype(int)),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            
            #mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
            # elbow, [original_width, original_height] -> normalized coordinates
        except Exception as e:
            print(f"Error: {e}")
            pass
        
        # Render detections
        # draw_landamarks -> utils to draw in image
        # results.pose_landmarks -> coordenates
        # mp_pose.POSE_CONNECTIONS -> how body points are conencted inside model
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        

        # Display the resized image
        cv2.imshow('UltiThrow', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()

107.94340661816396
106.90057468495392
100.16793586462508
98.32266100790989
90.91139506559237
91.08120677012758
102.99782799586123
95.54898322659973
98.00621974662533
118.15293888729305
107.6529017509565
120.8359845466983
117.62075715323407
102.49670522886305
128.32654235504543
127.96462893727912
133.33779754825034
133.6995235504813
135.45033870556946
129.54591481107406
136.56134531545925
118.6099583355641
101.94162963085586
112.04407043582472
112.7601300539089
108.52144513896287
104.09885124807228
96.67579675212302
90.28332988200698
97.54225653465176
98.936017490532
100.66594757644579
98.5924975850335
93.53234751885455
90.43465307040555
93.29086775392467
